<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>

### 데이터 준비 작업

In [1]:
import pickle
import pandas as pd

with open('titanic_step4_importance_train.pickle', 'rb') as pickle_filename:
    # 202402 업데이트: pandas 2.0.3 버전 이상에서는 pickle.load 실행시 에러가 남
    # 대신에 pd.read_pickle() 호출하면 됨    
    # train_importance = pickle.load(pickle_filename)
    train_importance = pd.read_pickle(pickle_filename)    
with open('titanic_step4_importance_test.pickle', 'rb') as pickle_filename:
    # 202402 업데이트: pandas 2.0.3 버전 이상에서는 pickle.load 실행시 에러가 남
    # 대신에 pd.read_pickle() 호출하면 됨    
    # test_importance = pickle.load(pickle_filename)    
    test_importance = pd.read_pickle(pickle_filename)
with open('titanic_step4_importance_train_y.pickle', 'rb') as pickle_filename:
    # 202402 업데이트: pandas 2.0.3 버전 이상에서는 pickle.load 실행시 에러가 남
    # 대신에 pd.read_pickle() 호출하면 됨    
    # train_answer = pickle.load(pickle_filename)    
    train_answer = pd.read_pickle(pickle_filename)

### 주요 라이브러리 임포트

In [2]:
import numpy as np # 각 모델에서 내부적으로 관련 라이브러리 사용 가능

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier             # 1. K-Nearest Neighbor(KNN)
from sklearn.linear_model import LogisticRegression            # 2. Logistic Regression
from sklearn.svm import SVC                                                # 3. SVC
from sklearn.tree import DecisionTreeClassifier                   # 4. Decision Tree
from sklearn.ensemble import RandomForestClassifier       # 5. Random Forest
from sklearn.ensemble import ExtraTreesClassifier             # 6. Extra Tree
from sklearn.ensemble import GradientBoostingClassifier  # 7. GBM
from sklearn.naive_bayes import GaussianNB                     # 8. GaussianNB
from xgboost import XGBClassifier                                     # 9. XGBoost
from lightgbm import LGBMClassifier                                 # 10. LightGBM
import warnings

warnings.filterwarnings('ignore')

### 모델 재트레닝

In [3]:
logreg_model = LogisticRegression(C=18.288277344191805, penalty='l2', random_state=1)

xgb_model = XGBClassifier(
    eval_metric = 'logloss',
    learning_rate=0.17, 
    n_estimators=10, 
    max_depth=6, 
    min_child_weight=1,
    gamma=0.2,
    reg_alpha=0.01,
    colsample_bytree=0.85,
    subsample=0.9,
    random_state=1
)

random_model = RandomForestClassifier(
    max_depth=None, 
    max_features=0.8, 
    min_samples_leaf=2, 
    min_samples_split=6, 
    n_estimators=200, 
    random_state=1
)

extra_model = ExtraTreesClassifier(
    max_depth=None, 
    max_features=0.5, 
    min_samples_leaf=2, 
    min_samples_split=10, 
    n_estimators=50, 
    random_state=1
)

### Voting Classifier
- 일종의 앙상블(ensemble) 의 또다른 기법3 으로(기법1은 Bagging, 기법2는 Boosting), 여러 모델들을 기반으로, 투표를 하는 Voting 기법이 있음
- 해당 기법을 사용하여 성능이 괜찮은 모델을 기반으로 Voting 을 해서, 또다른 예측 모델을 구성할 수 있음
- Voting 기법도 크게 Hard Voting 기법과 Soft Voting 기법이 존재함

### Hard Voting Classifier
- 여러 모델이 예측한 분류 중, 가장 많은 모델이 예측한 분류를 선택하는 기법

<img src="https://www.fun-coding.org/00_Images/hardvoting.png">

In [4]:
from sklearn.ensemble import VotingClassifier

grid_hard = VotingClassifier(estimators = [
        ('Logistic Regression', logreg_model),
        ('XGBoost', xgb_model),
        ('Random Forest', random_model), 
        ('Extra Trees', extra_model),
    ], voting = 'hard')

score = cross_val_score(grid_hard, train_importance, train_answer, cv=5, scoring='accuracy')
print(np.mean(score)*100)

84.28849413093967


### Soft Voting Classifier
- 여러 모델을 확률로 예측 분류한 후, 예측 확률의 평균을 내어, 확률이 가장 높은 분류를 선택하는 기법

<img src="https://www.fun-coding.org/00_Images/softvoting.png">

In [5]:
# 튜닝한 파라미터로 소프트보팅
grid_soft = VotingClassifier(estimators = [
        ('Logistic Regression', logreg_model),
        ('XGBoost', xgb_model),
        ('Random Forest', random_model), 
        ('Extra Trees', extra_model),
    ], voting = 'soft')

score = cross_val_score(grid_soft, train_importance, train_answer, cv=5, scoring='accuracy')
print(np.mean(score)*100)

83.95016006528152


### 최종 제출 테스트
> 타이타닉 경진대회는 외부에서, 테스트 데이터에 대한 정답셋을 구할 수 있으므로, 정답셋을 가지고 직접 테스트하기로 함

In [6]:
test = pd.read_csv('titanic/test.csv')
submission = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission['PassengerId'] = test['PassengerId']

In [7]:
submission.head()

,PassengerId,Survived
0,892,NaN
1,893,NaN
2,894,NaN
3,895,NaN
4,896,NaN


### 학습

In [8]:
grid_hard.fit(train_importance, train_answer)
#grid_soft.fit(train_importance, train_answer)

VotingClassifier(estimators=[('Logistic Regression',
                              LogisticRegression(C=18.288277344191805,
                                                 random_state=1)),
                             ('XGBoost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.85, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None, g...
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=10, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=1, ...)),
                             ('Random Forest',
                              RandomForestClassifier(max_features=0.8,
                                                     min_samples_leaf=2,
                                                     min_samples_split=6,
                                                     n_estimators=200,
                                                     random_state=1)),
                             ('Extra Trees',
                              ExtraTreesClassifier(max_features=0.5,
                                                   min_samples_leaf=2,
                                                   min_samples_split=10,
                                                   n_estimators=50,
                                                   random_state=1))])

### 제출

In [9]:
submission["Survived"] = grid_hard.predict(test_importance)
submission = submission.astype('int')
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [10]:
submission.to_csv('titanic_predict.csv', header = True, index = False)

In [11]:
!kaggle competitions submit -c titanic -f titanic_predict.csv -m "Test"

Traceback (most recent call last):
  File "/Users/davelee/opt_new/anaconda3/bin/kaggle", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Users/davelee/opt_new/anaconda3/lib/python3.11/site-packages/kaggle/cli.py", line 70, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/davelee/opt_new/anaconda3/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 801, in competition_submit_cli
    submit_result = self.competition_submit(file_name, message,
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/davelee/opt_new/anaconda3/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 748, in competition_submit
    url_result = self.process_response(
                 ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/davelee/opt_new/anaconda3/lib/python3.11/site-packages/kaggle/api/kaggle_api_extended.py", line 3565, in process_response
    raise Exception(data['message'])
Exception: Un

<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="font-weight:bold;color:#BF360C;">수고 많으셨습니다!</font><br>
<font size="4em" style="color:#BF360C;">Kaggle 타이타닉 랭킹: https://www.kaggle.com/c/titanic/leaderboard </font><br><br>  
<font size="4em" style="color:#BF360C;">정답셋을 외부에서 찾을 수 있으므로, 스코어가 1 에 가까운 값들은 모두 cheat 라고 봐야 하지만,</font><br>
    <font size="4em" style="color:#BF360C;">이러한 경우를 모두 포함하더라도 즉 <b>top 5%</b> 내의 스코어임</font>
</div>

<div class="alert alert-block" style="border: 2px solid #E65100;background-color:#FFF3E0;padding:10px">
<font size="4em" style="font-weight:bold;color:#BF360C;">큰그림으로 이해하기</font><br>
<font size="4em" style="color:#BF360C;">데이터 예측 기본 활용을 위해, 다음 단계를 패턴화해서, 활용하기</font><br><br>  
<font size="4em" style="color:#BF360C;">1. EDA</font><br>
<font size="4em" style="color:#BF360C;">2. Feature Engineering </font><br>    
<font size="4em" style="color:#BF360C;">&nbsp;&nbsp;&nbsp;&nbsp;- 숫자 변환 </font><br>    
<font size="4em" style="color:#BF360C;">&nbsp;&nbsp;&nbsp;&nbsp;- 도메인 이해 기반 영향력 있는 컬럼 추가 </font><br>       
<font size="4em" style="color:#BF360C;">&nbsp;&nbsp;&nbsp;&nbsp;- OneHot Encoding (또는 Label Encoding) </font><br>
<font size="4em" style="color:#BF360C;">&nbsp;&nbsp;&nbsp;&nbsp;<b>- 참고(추가 설명): 본 단계에서 스케일링 처리도 할 수 있으며, 이 부분은 이후 강의에서 설명드립니다</b></font><br>
<font size="4em" style="color:#BF360C;">&nbsp;&nbsp;&nbsp;&nbsp;- 중요도순 Feature Selection </font><br>                
<font size="4em" style="color:#BF360C;">3. Hyper Parameter Tuning (Random Search, Grid Search, Bayesian Optimization)</font><br>    
<font size="4em" style="color:#BF360C;">4. VotingClassifier 로 성능이 좋은 모델 기반 추가 모델까지 만든 후</font><br>    
<font size="4em" style="color:#BF360C;">5. 최종 훈련 및 예측</font><br>        
</div>

<div class="alert alert-block" style="border: 1px solid #455A64;background-color:#ECEFF1;">
본 자료 및 영상 컨텐츠는 저작권법 제25조 2항에 의해 보호를 받습니다. 본 컨텐츠 및 컨텐츠 일부 문구등을 외부에 공개, 게시하는 것을 금지합니다. 특히 자료에 대해서는 저작권법을 엄격하게 적용하겠습니다.
</div>